In [1]:
# References:
# https://github.com/speakleash/speakleash
# https://github.com/speakleash/speakleash-postprocesor/blob/main/postprocessor/analyzer.py
#https://github.com/mglabska/speakleash_filters/blob/master/quality.py

import numpy as np
import os
import pandas as pd
import pickle
import plotly.express as px
import spacy

from langdetect import detect
from speakleash import Speakleash
from wordcloud import WordCloud

In [2]:
%run dataframe.py

Enter a dataset name: forum_parenting_pl_corpus


100%|██████████████████████████████████████████████████████████████████████████████| 387M/387M [00:32<00:00, 11.9MiB/s]


In [3]:
with open(f".\\datasets\\forum_parenting_pl_corpus.pkl","rb") as f:
    df0 = pickle.load(f)

###### DataFrame

In [4]:
df0.head()

,url,sentences,words,verbs,nouns,punctuations,symbols,stopwords,oovs,characters,...,verb_ratio,adj_ratio,lexical_density,gunning_fog,pos_x,pos_num,camel_case,capitalized_words,quality,text
0,https://forum.parenting.pl/topic/23-witam-serd...,27,431,66,85,84,14,181,25,2763,...,0.1531,0.0835,0.5568,11.19,6,3,0,9,LOW,"Witam na forum, jestem Mamą dwójki dzieci: prz..."
1,https://forum.parenting.pl/topic/24-mm-2005-za...,100381,1475263,226849,325429,282264,16059,642962,91799,9178263,...,0.1538,0.0784,0.0591,7.18,18962,13332,3672,25677,LOW,Marcóweczki :) czekam tu na chętne do pogawęde...
2,https://forum.parenting.pl/topic/25-sprawdzam/,5,64,13,11,20,8,28,3,407,...,0.2031,0.0156,0.7812,9.77,0,0,1,2,LOW,"Sprawdzam jak mnie widać :)\nJa też, zawsze ma..."
3,https://forum.parenting.pl/topic/30-ja-rowniez...,4,36,8,5,8,2,19,0,228,...,0.2222,0.0278,0.8333,8.31,0,0,0,0,LOW,Witam wszystkich przyszłych użytkowników tego ...
4,https://forum.parenting.pl/topic/32-witam-bard...,7,53,13,11,21,6,18,3,380,...,0.2453,0.1509,0.8491,8.82,0,0,0,0,LOW,"Bardzo się cieszę, że forum pomału zaczyna dzi..."


In [5]:
df0.columns

Index(['url', 'sentences', 'words', 'verbs', 'nouns', 'punctuations',
       'symbols', 'stopwords', 'oovs', 'characters', 'avg_sentence_length',
       'adverbs', 'adjectives', 'avg_word_length', 'noun_ratio', 'verb_ratio',
       'adj_ratio', 'lexical_density', 'gunning_fog', 'pos_x', 'pos_num',
       'camel_case', 'capitalized_words', 'quality', 'text'],
      dtype='object')

###### Duplikaty

In [6]:
df0[df0['text'].isna()]

,url,sentences,words,verbs,nouns,punctuations,symbols,stopwords,oovs,characters,...,verb_ratio,adj_ratio,lexical_density,gunning_fog,pos_x,pos_num,camel_case,capitalized_words,quality,text


In [7]:
df0[df0['text'].duplicated()]

,url,sentences,words,verbs,nouns,punctuations,symbols,stopwords,oovs,characters,...,verb_ratio,adj_ratio,lexical_density,gunning_fog,pos_x,pos_num,camel_case,capitalized_words,quality,text
2197,https://forum.parenting.pl/topic/1035000-niani...,4,63,8,21,9,0,14,6,480,...,0.1270,0.1270,0.8730,14.57,1,3,0,6,LOW,Firma TOPCOM wprowadza nianię Babytalker 3500 ...
2339,https://forum.parenting.pl/topic/1073720-jak-p...,4,37,8,6,6,0,15,0,265,...,0.2162,0.1081,0.8919,9.13,0,0,0,0,LOW,Pojawiają się nagle i niespodziewanie. Często ...
2483,https://forum.parenting.pl/topic/1107538-niani...,4,63,8,21,9,0,14,6,480,...,0.1270,0.1270,0.8730,14.57,1,3,0,6,LOW,Firma TOPCOM wprowadza nianię Babytalker 3500 ...
3329,https://forum.parenting.pl/topic/1334129-badz-...,5,72,10,22,13,0,22,2,525,...,0.1389,0.0833,0.9028,10.20,1,0,0,5,LOW,"Doktor Jadwiga Górnicka, znana polska lekarka ..."
3843,https://forum.parenting.pl/topic/1465531-dziec...,3,63,8,22,9,0,20,2,504,...,0.1270,0.1270,0.8889,16.02,1,0,0,0,LOW,Troska o kręgosłup dziecka powinna zacząć się ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28472,https://forum.parenting.pl/topic/4665377-mantr...,44,1135,25,334,130,0,37,228,8266,...,0.0220,0.0758,0.3339,13.91,254,8,0,10,LOW,Consult Now +91 8769179991\n\n\n\twebsite http...
28494,https://forum.parenting.pl/topic/4667949-jual-...,23,277,13,46,34,1,1,182,1837,...,0.0469,0.0614,0.5776,8.08,45,2,0,28,LOW,JUAL OBAT ABORSI ASLI 083816989428 MELAYANI PE...
28500,https://forum.parenting.pl/topic/4668444-acqui...,30,664,33,158,142,0,14,313,5343,...,0.0497,0.1069,0.4413,14.21,163,10,0,23,LOW,"ACQUISTA VERA PATENTE DI GUIDA TEDESCA, PASSAP..."
28501,https://forum.parenting.pl/topic/4668445-acqui...,30,664,33,158,142,0,14,313,5343,...,0.0497,0.1069,0.4413,14.21,163,10,0,23,LOW,"ACQUISTA VERA PATENTE DI GUIDA TEDESCA, PASSAP..."


In [8]:
%run charts.py

Enter a dataset name: forum_parenting_pl_corpus


###### WordCloud

In [14]:
# tokenizer i lemmatizer wymagają więcej pamięci, dlatego tokenizacja przez .split()

nlp = spacy.load('pl_core_news_md')
nlp.max_length = 1000000
all_stopwords = nlp.Defaults.stop_words
df0['clean'] = df0['text'].apply(
    lambda x: " ".join(word for word in x.split() if word.lower() not in all_stopwords and word.isalpha())
)

In [15]:
clean_text =  " ".join(df0['clean'].values)
wordcloud = WordCloud().generate(clean_text)
wordcloud.to_file('wordcloud.jpg')

In [20]:
fig = px.bar(x = wordcloud.words_.keys(), y = wordcloud.words_.values())
fig.write_html(f"imgs/freq_bar.html")

In [12]:
def detect_lang(df):    
    df_l = df.copy()
    df_l['lang'] = df_l['text'].apply(lambda x: detect(x))
    return df_l

df_l = detect_lang(df0)

In [13]:
with open("to_train.pkl","wb") as f:
    pickle.dump(df_l, f)